# Markdown cell


In [7]:
!pip install --quiet streamlit langchain google-cloud-aiplatform langchain-google-vertexai google-cloud-vertexai langchain-community faiss-cpu

In [ ]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8502 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

In [8]:
%%writefile SWMChat.py
import streamlit as st
import langchain as lg
st.set_page_config(page_title="Aplicación Streamlit", page_icon="⚙️", layout="centered")

st.title("Normativas de natación")
prompt = st.chat_input("Say something")
if prompt:
    st.write(f"User has sent the following prompt: {prompt}")

Overwriting SWMChat.py


In [4]:
!streamlit run SWMChat.py --server.port 8502

^C
